In [1]:
from models.WavePattern import WavePattern
from models.WaveRules import Impulse, LeadingDiagonal, Correction
from models.WaveAnalyzer import WaveAnalyzer
from models.WaveCycle import WaveCycle
from models.WaveOptions import WaveOptionsGenerator5, WaveOptionsGenerator3
from models.helpers import plot_pattern
from models.helpers import plot_cycle
from models.helpers import plot_pattern_two

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [3]:
global df
df_data = pd.read_csv(r'./data/BTC-USD.csv') # 5 years
df = df_data.iloc[len(list(df_data.Date)) - 365 :, :] # 1 year plotting
idx_start = np.argmin(np.array(list(df['Low'])))


In [4]:
wa = WaveAnalyzer(df=df, verbose=False)
wave_options_impulse = WaveOptionsGenerator5(up_to=15)  # generates WaveOptions up to [15, 15, 15, 15, 15]
wave_options_correction = WaveOptionsGenerator3(up_to=9) 

In [5]:
impulse = Impulse('impulse')
leading_diagonal = LeadingDiagonal('leading diagonal')
correction = Correction('correction')
rules_to_check = [impulse, correction]

In [6]:
print(f'Start at idx: {idx_start}')
print(f"will run up to {wave_options_impulse.number / 1e6}M combinations.")

Start at idx: 78
will run up to 0.579195M combinations.


In [7]:
wavepatterns_up = list()
wavepatterns_down = list()
completeList = list()

In [8]:
try:
    for new_option_impulse in wave_options_impulse.options_sorted:

        waves_up = wa.find_impulsive_wave(idx_start=idx_start, wave_config=new_option_impulse.values)

        if waves_up:
            wavepattern_up = WavePattern(waves_up, verbose=True)

            for rule in rules_to_check:

                if wavepattern_up.check_rule(rule):
                    if wavepattern_up in wavepatterns_up:
                        continue
                    else:
                        wavepatterns_up.append(wavepattern_up)
                        print(f'{rule.name} found: {new_option_impulse.values}')
                        cor_end = waves_up[4].idx_end
                        cor_date = waves_up[4].date_end
                        cor_close = waves_up[4].high
                        cor_low = waves_up[4].low
except Exception as e:
    print("\n\n\n",e)
    

print(cor_end) 

Rule Violation of impulse for condition w4_1: End of Wave4 is lower than End of Wave1
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w3_2: End of Wave3 is lower than End of Wave1
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w3_1: Wave3 is the shortest Wave.
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w4_1: End of Wave4 is lower than End of Wave1
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
impulse found: [1, 3, 2, 0, 0]
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
impulse found: [1, 3, 3, 0, 0]
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
impulse found: [1, 3, 4, 0, 0]
Rule Violation of

In [11]:
wa_pred = WaveAnalyzer(df=df, verbose=False)
wavepatterns_down_pred = []
wave_options_correction_pred = WaveOptionsGenerator3(up_to=9)
idx_start_cor = cor_end
try:
    wave_cycles = set()
    for new_option_correction_pred in wave_options_correction_pred.options_sorted:
        waves_cor_pred = wa_pred.find_corrective_wave(idx_start=idx_start_cor, wave_config=new_option_correction_pred.values)

        if waves_cor_pred:
            wavepattern_cor_pred = WavePattern(waves_cor_pred, verbose=True)

            for rule in rules_to_check:

                if wavepattern_cor_pred.check_rule(rule):
                    if wavepattern_cor_pred in wavepatterns_down_pred:
                        continue
                    else:
                        wavepatterns_down_pred.append(wavepattern_cor_pred)
                        # print(f'{rule.name} found: {new_option_correction_pred.values}')
    
except Exception as e:
    print("\n\n\n", e)
plot_pattern_two(df=df, wave_pattern=wavepattern_cor_pred, title="Corrective Wave")
# print(df_pred)
wavepatterns_down_pred

Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_5: WaveC shorter (value wise) than 0.60 x WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_5: WaveC shorter (value wise) than 0.60 x WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_2: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_2: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
R